## Índice

1. [Introducción y objetivos](#1-introducción-y-objetivos)
2. [Carga de datos y preparación](#2-carga-de-datos-y-preparación)
3. [Extracción y limpieza de texto](#3-extracción-y-limpieza-de-texto)
    - 3.1. Ejemplo de estructura del PDF
    - 3.2. Detección y eliminación de ruido
    - 3.3. Análisis de títulos y secciones
4. [Generación de chunks](#4-generación-de-chunks)

---

## 1. Introducción y objetivos

En este notebook se va a extraer y limpiar texto de documentos normativos en PDF para montar un corpus usable en tareas de recuperación de información. El objetivo es dejar los datos listos para probar chunking y modelos de RAG sobre casos reales, no solo datasets de ejemplo.

---

## 2. Carga de datos y preparación

Se cargan los PDFs en bruto desde la carpeta del proyecto.

In [1]:
import os

pdf_dir = "../data/raw_pdfs"
pdf_files = [f for f in os.listdir(pdf_dir) if f.lower().endswith(".pdf")]
pdf_files

['ai_hleg_ethics_guidelines.pdf',
 'eu_ai_act_regulation.pdf',
 'nist_privacy_framework_v1.pdf',
 'oecd_ai_classification_framework.pdf',
 'oecd_legal_0449_en.pdf']

## 3. Extracción y limpieza de texto

Se empieza la extracción del texto de los PDFs para ver la estructura y el tipo de contenido que tiene cada uno. El objetivo aquí es detectar qué ruido hay (como encabezados, pies de página o tablas raras) y ver si los títulos y secciones se pueden distinguir bien.

De manera manual, se han revisado por encima los PDFs y se quitan las sigiuentes páginas:

| PDF                           | Páginas útiles       |
|-------------------------------|---------------------|
| oecd_legal_0449_en.pdf        | 3 a 9               | 
| eu_ai_act_regulation.pdf      | 2 a 95              | 
| oecd_ai_classification_framework.pdf | 16 a 67       |
| ai_hleg_ethics_guidelines.pdf | 4 a 37              |
| nist_privacy_framework_v1.pdf | 5 a 43              |


In [3]:
import fitz  # PyMuPDF

# zero-based index
pages_dict = {
    "oecd_legal_0449_en.pdf": range(2, 9),
    "eu_ai_act_regulation.pdf": range(1, 95),
    "oecd_ai_classification_framework.pdf": range(15, 67),
    "ai_hleg_ethics_guidelines.pdf": range(3, 37),
    "nist_privacy_framework_v1.pdf": range(4, 43),
}

for pdf_file in pdf_files:
    if pdf_file not in pages_dict:
        continue
    pdf_path = os.path.join(pdf_dir, pdf_file)
    doc = fitz.open(pdf_path)
    page_idxs = list(pages_dict[pdf_file])
    print(f"\n--- {pdf_file} ({len(page_idxs)} páginas útiles) ---")

    for idx, pagina_idx in enumerate([0, 2]):
        if pagina_idx >= len(page_idxs):
            continue
        p = page_idxs[pagina_idx]
        page = doc.load_page(p)
        print(f"\n--- Página real {p+1} (índice útil {pagina_idx+1}) ---")
        print(page.get_text())



--- ai_hleg_ethics_guidelines.pdf (34 páginas útiles) ---

--- Página real 4 (índice útil 1) ---
2 
 
EXECUTIVE SUMMARY 
The aim of the Guidelines is to promote Trustworthy AI. Trustworthy AI has three components, which should be 
met throughout the system's entire life cycle: (1) it should be lawful, complying with all applicable laws and 
regulations (2) it should be ethical, ensuring adherence to ethical principles and values and (3) it should be robust, 
both from a technical and social perspective since, even with good intentions, AI systems can cause unintentional 
harm. Each component in itself is necessary but not sufficient for the achievement of Trustworthy AI. Ideally, all 
three components work in harmony and overlap in their operation. If, in practice, tensions arise between these 
components, society should endeavour to align them.  
These Guidelines set out a framework for achieving Trustworthy AI. The framework does not explicitly deal with 
Trustworthy AI’s first comp

Se guarda el texto y los metadatos de cada página útil por separado. Así luego es fácil saber de dónde sale cada fragmento cuando se hagan los chunks, aunque todavía no esté decidido el tamaño o la forma del chunking.


In [4]:
page_records = []

for pdf_file in pdf_files:
    if pdf_file not in pages_dict:
        continue
    pdf_path = os.path.join(pdf_dir, pdf_file)
    doc = fitz.open(pdf_path)
    page_idxs = list(pages_dict[pdf_file])
    for i, p in enumerate(page_idxs):
        page = doc.load_page(p)
        text = page.get_text()
        page_records.append({
            "pdf": pdf_file,
            "page_real": p + 1,  # página dentro del pdf
            "page_util": i + 1,  # posición dentro de las útiles
            "text": text
        })

In [5]:
# 3 ejemplos
for i, record in enumerate(page_records[:3]):
    print(f"Ejemplo {i+1}:")
    print(f"  PDF: {record['pdf']}")
    print(f"  Página real: {record['page_real']}")
    print(f"  Página útil: {record['page_util']}")
    print(f"  Texto (primeros 300 caracteres):\n{record['text'][:300]}")
    print("-" * 60)


Ejemplo 1:
  PDF: ai_hleg_ethics_guidelines.pdf
  Página real: 4
  Página útil: 1
  Texto (primeros 300 caracteres):
2 
 
EXECUTIVE SUMMARY 
The aim of the Guidelines is to promote Trustworthy AI. Trustworthy AI has three components, which should be 
met throughout the system's entire life cycle: (1) it should be lawful, complying with all applicable laws and 
regulations (2) it should be ethical, ensuring adheren
------------------------------------------------------------
Ejemplo 2:
  PDF: ai_hleg_ethics_guidelines.pdf
  Página real: 5
  Página útil: 2
  Texto (primeros 300 caracteres):
3 
 
 Foster research and innovation to help assess AI systems and to further the achievement of the 
requirements; disseminate results and open questions to the wider public, and systematically train a new 
generation of experts in AI ethics. 
 Communicate, in a clear and proactive manner, inform
------------------------------------------------------------
Ejemplo 3:
  PDF: ai_hleg_ethics_guideline

In [ ]:
total_words = sum(len(record["text"].split()) for record in page_records)
print(f"Total de palabras en todas las páginas útiles: {total_words}")

for i, record in enumerate(page_records[:5]):
    n_words = len(record["text"].split())
    print(f"{record['pdf']} | Página real: {record['page_real']} | Palabras: {n_words}")


Total de palabras en todas las páginas útiles: 107749
ai_hleg_ethics_guidelines.pdf | Página real: 4 | Palabras: 632
ai_hleg_ethics_guidelines.pdf | Página real: 5 | Palabras: 590
ai_hleg_ethics_guidelines.pdf | Página real: 6 | Palabras: 712
ai_hleg_ethics_guidelines.pdf | Página real: 7 | Palabras: 652
ai_hleg_ethics_guidelines.pdf | Página real: 8 | Palabras: 674


Se procede a:

Eliminar líneas vacías.

Quitar líneas que son solo números (números de página).

Quitar líneas con solo espacios.

In [7]:
for record in page_records:
    lines = record["text"].split("\n")
    clean_lines = []
    for line in lines:
        l = line.strip()
        if not l:
            continue  # línea vacía
        if l.isdigit():
            continue  # solo número
        clean_lines.append(l)
    record["text_clean"] = "\n".join(clean_lines)


In [ ]:
from collections import Counter, defaultdict

palabras_por_pdf = defaultdict(int)
all_lines = []
posibles_headers = Counter()
posibles_titulos = []
posibles_tablas = []
paginas_vacias = []
lineas_con_urls = []
lineas_con_caracteres_raros = []

for record in page_records:
    pdf = record["pdf"]
    page = record["page_real"]
    text = record["text_clean"]
    n_words = len(text.split())
    palabras_por_pdf[pdf] += n_words

    # Separar por líneas
    lines = [l.strip() for l in text.split("\n") if l.strip()]
    all_lines.extend(lines)

    # Headers/footers candidatos: primeras y últimas líneas
    if lines:
        posibles_headers[lines[0]] += 1
        posibles_headers[lines[-1]] += 1

    # Títulos/secciones: mayúsculas o numeración tipo "1.", "2.1", etc.
    for line in lines:
        if line.isupper() or line.startswith(tuple(f"{i}." for i in range(10))):
            posibles_titulos.append((pdf, page, line))
        # Tablas o líneas sospechosas
        if "|" in line or "----" in line or line.count("  ") > 3:
            posibles_tablas.append((pdf, page, line))
        # URLs/enlaces
        if "http" in line or "www." in line:
            lineas_con_urls.append((pdf, page, line))
        # Caracteres no ASCII
        if any(ord(char) > 127 for char in line):
            lineas_con_caracteres_raros.append((pdf, page, line))

    # Páginas vacías
    if n_words < 20:
        paginas_vacias.append((pdf, page, n_words))

# Resultados principales

print("\n--- Palabras totales por documento ---")
for pdf, n in palabras_por_pdf.items():
    print(f"{pdf}: {n} palabras")

print("\n--- Headers/footers más repetidos (top 10) ---")
for header, count in posibles_headers.most_common(10):
    print(f"'{header}' aparece {count} veces")

print("\n--- Ejemplos de títulos o secciones detectados (top 10) ---")
for t in posibles_titulos[:10]:
    print(f"{t[0]} | Página {t[1]}: {t[2]}")

print("\n--- Ejemplos de posibles tablas o líneas sospechosas (top 5) ---")
for t in posibles_tablas[:5]:
    print(f"{t[0]} | Página {t[1]}: {t[2]}")

print("\n--- Páginas casi vacías (<20 palabras) ---")
for v in paginas_vacias:
    print(f"{v[0]} | Página {v[1]} | Palabras: {v[2]}")

print("\n--- Ejemplos de líneas con URLs (top 5) ---")
for u in lineas_con_urls[:5]:
    print(f"{u[0]} | Página {u[1]}: {u[2]}")

print("\n--- Ejemplos de líneas con caracteres raros (top 5) ---")
for r in lineas_con_caracteres_raros[:5]:
    print(f"{r[0]} | Página {r[1]}: {r[2]}")



--- Palabras totales por documento ---
ai_hleg_ethics_guidelines.pdf: 20480 palabras
eu_ai_act_regulation.pdf: 44606 palabras
nist_privacy_framework_v1.pdf: 14465 palabras
oecd_ai_classification_framework.pdf: 24491 palabras
oecd_legal_0449_en.pdf: 3269 palabras

--- Headers/footers más repetidos (top 10) ---
'EN' aparece 94 veces
'NIST Privacy Framework' aparece 39 veces
'' aparece 3 veces
'Figure 1: The Guidelines as a framework for Trustworthy AI' aparece 2 veces
'surrender procedures between Member States (OJ L 190, 18.7.2002, p. 1).' aparece 2 veces
'EXECUTIVE SUMMARY' aparece 1 veces
'elderly, the integration of persons with disabilities and workers’ rights. See also article 38 dealing with consumer protection.' aparece 1 veces
' Foster research and innovation to help assess AI systems and to further the achievement of the' aparece 1 veces
'published in December 2018 by the Commission and Member States.' aparece 1 veces
'A. INTRODUCTION' aparece 1 veces

--- Ejemplos de título

> Se aplican reglas extra de limpieza:
> - Se eliminan los headers y footers repetidos que se han detectado (como 'EN' y 'NIST Privacy Framework').
> - Los títulos que encajan con patrones se guardan como metadato 'titulo_interpretado' para cada página.
> - Se omiten páginas casi vacías, ya que suelen ser separadores o finales de sección.
> - Se eliminan las líneas que solo son URLs, porque no se enlazará a fuentes externas.
> - Se sustituyen los caracteres raros (no ASCII) por espacios para que el texto quede limpio pero no se pierda info útil.

In [16]:
import re

# Headers y footers a eliminar
headers_a_eliminar = {"EN", "NIST Privacy Framework"}

def es_url(line):
    return "http" in line or "www." in line

def tiene_caracter_raro(line):
    return any(ord(char) > 127 for char in line)

def limpiar_caracteres_raros(line):
    return ''.join(char if ord(char) < 128 else ' ' for char in line)

nuevos_records = []
for record in page_records:
    lines = [l.strip() for l in record["text"].split("\n") if l.strip()]
    clean_lines = []
    titulo_interpretado = None
    for i, line in enumerate(lines):
        if line in headers_a_eliminar:
            continue
        if es_url(line):
            continue
        if line.isdigit():
            continue
        line_limpia = limpiar_caracteres_raros(line)
        # Detección de títulos
        if (line_limpia.isupper() or re.match(r"^[A-Z]\.\s", line_limpia) or re.match(r"^\d+(\.\d+)*", line_limpia)):
            if titulo_interpretado is None:
                titulo_interpretado = line_limpia
        clean_lines.append(line_limpia)
    texto_final = "\n".join(clean_lines)
    if len(texto_final.split()) < 20:
        continue
    nuevos_records.append({
        "pdf": record["pdf"],
        "page_real": record["page_real"],
        "page_util": record["page_util"],
        "titulo_interpretado": titulo_interpretado,
        "text_clean": texto_final
    })

# Reemplaza page_records por los nuevos
page_records = nuevos_records


In [17]:
titulos_por_pdf = defaultdict(list)
for record in page_records:
    if record["titulo_interpretado"]:
        titulos_por_pdf[record["pdf"]].append((record["page_real"], record["titulo_interpretado"]))

# índice de cada PDF
for pdf, titulos in titulos_por_pdf.items():
    print(f"\n=== {pdf} ===")
    for page, titulo in titulos:
        print(f"  Página {page}: {titulo}")



=== ai_hleg_ethics_guidelines.pdf ===
  Página 4: EXECUTIVE SUMMARY
  Página 6: A. INTRODUCTION
  Página 7: 1. it should be lawful, complying with all applicable laws and regulations;
  Página 8: B. A FRAMEWORK FOR TRUSTWORTHY AI
  Página 11: I.
  Página 12: 2. From fundamental rights to ethical principles
  Página 13: 2.2 Ethical Principles in the Context of AI Systems22
  Página 15: 2.3 Tensions between the principles
  Página 16: II.
  Página 17: 1.1 Human agency and oversight
  Página 18: 1.2 Technical robustness and safety
  Página 19: 1.3 Privacy and data governance
  Página 20: 1.4 Transparency
  Página 21: 1.6 Societal and environmental well-being
  Página 22: 2. Technical and non-technical methods to realise Trustworthy AI
  Página 23: 2.1. Technical methods
  Página 24: 2.2. Non-technical methods
  Página 25: IEEE
  Página 27: HR
  Página 28: TRUSTWORTHY AI ASSESSMENT LIST (PILOT VERSION)
  Página 29: 2. Technical robustness and safety
  Página 30: 3. Privacy and data govern

Muchos de los títulos detectados no aportan mucha información, ya que algunos son solo números o referencias de página. Aunque no son muy útiles por sí mismos, los dejamos en los metadatos porque ocupan poco y pueden servir como referencia extra si hace falta luego.

---

## 4. Generación de chunks

In [18]:
chunk_size = 300

chunks = []
for pdf in set(r["pdf"] for r in page_records):
    # Filtra las páginas de este PDF y las ordena
    paginas = [r for r in page_records if r["pdf"] == pdf]
    paginas.sort(key=lambda x: x["page_real"])
    
    palabras = []
    paginas_chunk = []
    titulos_chunk = set()
    idx_chunk = 1
    
    for p in paginas:
        palabras_pagina = p["text_clean"].split()
        if p["titulo_interpretado"]:
            titulos_chunk.add(p["titulo_interpretado"])
        paginas_chunk.append(p["page_real"])
        
        for w in palabras_pagina:
            palabras.append(w)
            if len(palabras) == chunk_size:
                chunk_text = " ".join(palabras)
                chunks.append({
                    "pdf": pdf,
                    "pages": paginas_chunk.copy(),
                    "titles": list(titulos_chunk),
                    "chunk_index": idx_chunk,
                    "text": chunk_text,
                    "n_words": len(palabras)
                })
                idx_chunk += 1
                palabras = []
                paginas_chunk = []
                titulos_chunk = set()
    
    if palabras:
        chunk_text = " ".join(palabras)
        chunks.append({
            "pdf": pdf,
            "pages": paginas_chunk.copy(),
            "titles": list(titulos_chunk),
            "chunk_index": idx_chunk,
            "text": chunk_text,
            "n_words": len(palabras)
        })

In [24]:
import os
import json

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
output_dir = os.path.join(project_root, "data", "chunks")
os.makedirs(output_dir, exist_ok=True)

output_file = os.path.join(output_dir, "corpus_chunks_300w.json")

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)

print(f"Guardado el corpus en {output_file}")




Guardado el corpus en d:\TFM_RAG_NOR\data\chunks\corpus_chunks_300w.json


In [25]:
import random

# chunks en total y por documento
from collections import Counter

print(f"Total de chunks: {len(chunks)}")
chunks_por_pdf = Counter(chunk["pdf"] for chunk in chunks)
for pdf, n in chunks_por_pdf.items():
    print(f"{pdf}: {n} chunks")

# distribución de palabras por chunk
palabras_por_chunk = [chunk["n_words"] for chunk in chunks]
print(f"\nPalabras por chunk: min={min(palabras_por_chunk)}, max={max(palabras_por_chunk)}, media={sum(palabras_por_chunk)//len(palabras_por_chunk)}")

# metadatos
print("\nEjemplos aleatorios de chunks:\n")
for i in random.sample(range(len(chunks)), min(5, len(chunks))):
    c = chunks[i]
    print(f"PDF: {c['pdf']}")
    print(f"Chunk #{c['chunk_index']}, Páginas: {c['pages']}, Títulos: {c['titles']}")
    print(f"Nº palabras: {c['n_words']}")
    print(f"Texto (primeros 200 caracteres):\n{c['text'][:200]}")
    print("-" * 80)


Total de chunks: 357
oecd_ai_classification_framework.pdf: 81 chunks
ai_hleg_ethics_guidelines.pdf: 68 chunks
nist_privacy_framework_v1.pdf: 48 chunks
eu_ai_act_regulation.pdf: 149 chunks
oecd_legal_0449_en.pdf: 11 chunks

Palabras por chunk: min=74, max=300, media=298

Ejemplos aleatorios de chunks:

PDF: eu_ai_act_regulation.pdf
Chunk #92, Páginas: [54], Títulos: ['1.']
Nº palabras: 300
Texto (primeros 200 caracteres):
AI systems shall: (a) ensure that their high-risk AI systems are compliant with the requirements set out in Chapter 2 of this Title; (b) have a quality management system in place which complies with A
--------------------------------------------------------------------------------
PDF: ai_hleg_ethics_guidelines.pdf
Chunk #44, Páginas: [], Títulos: []
Nº palabras: 300
Texto (primeros 200 caracteres):
or board, is to provide oversight and advice. As set out above, certification specifications and bodies can also play a role to this end. Communication channels should be e